# Consulta 05

Esta consulta devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de rese;as de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo, en caso de no estar categorizados o encontrarse arroja el mensaje "No se encontró información sobre el desarrollador '...'".

Ejemplo de retorno: {'Valve' : [Negative = 182, Positive = 278]}



*Importar las librerías necesarias para el procesamiento de los datos*

In [1]:
import pandas as pd


*Importamos los archivos con los que vamos a trabajar la consulta*

In [2]:
games = pd.read_parquet(r'C:\Users\Owner\Desktop\LABS\Proyecto_STEAM\datos_STEAM\parquet\games_clean.parquet')
sentiment = pd.read_parquet(r'C:\Users\Owner\Desktop\LABS\Proyecto_STEAM\datos_STEAM\parquet\reviews_clean_sentiment.parquet')

*Realizamos una copia para trabajar sobre ella y mantener los datos intactos en el archivo original.*

In [3]:
games_copy = games.copy()
sentiment_copy = sentiment.copy()

*Realizamos las siguientes acciones:*
1. Combinar conjuntos de datos en las columnas apropiadas ('item_id' en reviews y 'id' en games).
2. Filtrar las filas donde el puntaje de sentimiento es positivo (2) o negativo (0), excluyendo los sentimientos neutrales.
3. Agrupar los datos filtrados por desarrollador y puntaje de sentimiento, contando la cantidad de reseñas.
4. Verificar si el desarrollador está presente en el DataFrame y maneja excepciones si no lo está.
5. Extraer la cantidad de reseñas positivas y negativas para el desarrollador especificado.
6. Convertir las cantidades en un formato de lista con claves especificadas.
7. Retornar un diccionario con las cantidades de reseñas positivas y negativas para el desarrollador dado. Si el desarrollador no se encuentra en los datos, devuelve un mensaje indicando que no se encontró información sobre ese desarrollador.

In [4]:
def developer_reviews_analysis(desarrollador):
    # Combinar conjuntos de datos en las columnas apropiadas ('item_id' en reviews y 'id' en games)
    merged_data = pd.merge(sentiment_copy, games_copy, left_on='item_id', right_on='id')
    
    # Filtrar filas donde el puntaje de sentimiento es positivo (2) o negativo (0)
    filtered_data = merged_data[merged_data['sentiment_analysis'] != 1]  # Excluir sentimiento neutral
    
    # Agrupar por desarrollador y puntaje de sentimiento, contar la cantidad de resenas
    grouped_data = filtered_data.groupby(['developer', 'sentiment_analysis']).size().unstack(fill_value=0)
    
    # Verificar si el desarrollador está en el DataFrame para manejar excepciones
    if desarrollador in grouped_data:
        # Extraer cantidad de resenas positivas y negativas para la desarrollador especificada
        developer_reviews = grouped_data.loc[desarrollador]
    
        # Convertir cantidades a formato de lista con claves especificadas
        developer_reviews_list = [
            {"Negativas": developer_reviews.get(0, 0)},
            {"Positivas": developer_reviews.get(2, 0)}
        ]
    
        return {desarrollador: developer_reviews_list}
    else:
        return f"No se encontró información sobre el desarrollador {desarrollador}"


*Aplicamos la función y corroboramos que funcione correctamente*

In [5]:
# Example usage:
developer = input("Ingrese nombre del desarrollador: ")
resultado = developer_reviews_analysis(developer)
print(resultado)

No se encontró información sobre el desarrollador Valve
